In [12]:
import json
from unicodedata import numeric
import numpy as np
import pandas as pd
import re
# A function is created to clean the data within file
def clean(x):
    x=str(x)
    x=x.replace("&nbsp;", " ")
    x=x.replace(". Vs. ", " v ")
    x=x.replace("\t", "   ")
    x=x.replace(":-", "")
    x=x.replace(";", " ")
    x=x.replace("&#39;", "'")
    x=x.replace("&amp;", "&")
    return x

In [11]:
#data is cleaned
data=pd.read_json("processed.json")
data=data.applymap(clean)

df=pd.DataFrame(data)
df.head(1)

,CaseNo,Judgement,ArgumentsOfPetitioner,ArgumentsOfResponder,Facts,Principle,Statute,CaseCited,Reasoning,Decision,NoOfJudges,Decision_Status
0,Civil Appeal No. 6006 Of 2001 (With C.A. No. 3...,Dharmadhikari J. These two cross appeals ...,32. It needs to be mentioned at this stage tha...,44. The alternative argument advanced on behal...,1.These two cross appeals have been preferred ...,1.These two cross appeals have been preferred ...,"Specific Relief Act, 1963 Section 20",Lala Durga Prasad vs. Lala Deep Chand. (1954 S...,After hearing the argument at length advanced ...,70. On duly appreciating of the evidence on r...,2,2


My approach says that the plantiff starts with a disadvantage considering how in any given case
the defendant is "Innocent until proven guilty", this disadvantage will be shown on the basis of 
the cases cited in the current case where the decision would lead to the disadvantage score being
higher or lower based on who won the cases cited and how similar they were, getting how similar they 
were is an issue but can be probably solved with an NLP, multiplying the similarity grading from 0-1 
based on how similar they were to the case with 100 (100 being the standard value for disadvantage in case of no similarity), and adding that score or subtracting it from
0 (based on who won),  giving us the final disadvantage score, so if two cases are exactly similar we can already tell that the previous decision is
going to be referenced to.

In [3]:
similarity=0.4 #similarity to previous cited case;
side="Plantiff" #side that won cited case
disadvantage_quotient=100*similarity
if (side=="Plantiff"):
    Disadvantage= 100-disadvantage_quotient
else:
    Disadvantage=100+disadvantage_quotient
if Disadvantage<=0:
    print("Plantiff wins the case") #in case the plantiff reaches a disadvantage of 0, means the defendant has lost said case

we then consider each argument , each argument is given an impact which it has on the case,
This is under the assumption that all arguments that the sides have given have some impact on said case
We then have to consider how much impact an argument would have on the case , this being the difficult part
Firstly we need to seperate each argument from one another.

In [4]:
pattern=r"\d{2}[.]\s"
print(re.search(pattern,"22. op weqaw"))

<re.Match object; span=(0, 4), match='22. '>


In [10]:
Arguments_of_Petitioner=re.split(pattern,df.iloc[0,2])
Arguments_of_Petitioner[1] #Argument number 1 from the list of the arguments of the first case

'It needs to be mentioned at this stage that learned counsel appearing for the subsequent vendee has not disputed in this appeal that the sale in favour of the subsequent vendee was with notice of the prior sale agreement Ex. P1 with the vendee.'

In [16]:
Arguments_of_Defendant=re.split(pattern,df.iloc[0,3])
Arguments_of_Defendant[1]#argument number 1 from the arguments of defendant list

'The alternative argument advanced on behalf of the plaintiff-vendor is that although the petitioner is, in law, entitled to conveyance of full title in the property by the vendor, the reversioners and the subsequent vendee, he has restricted his claim to the lesser relief of seeking conveyance only of life interest in the property of the vendor. Such relief can be granted under section 12(3) of the Specific Relief Act as the vendee is willing to pay full agreed consideration for lesser relief of conveyance of life interest in the property. Reliance is placed on Lala Durga Prasad vs. Lala Deep Chand. (1954 SCR 360 at page. 367)  Jhumma Masjid Vs. Kodimaniandra Devaiah (1962 Supp (2) SCR 554 at pg. 570)  Soni Lalji Jetha vs. Sonkalidas Devchand (1967(1) SCR 873 at pg. 879)  and Narandas Karsondas vs. S.A. Kamtam (1977(3) SCC 247).'

The arguments can be identified with the following regix pattern from what I can see
The arguments are then compared to the facts and check if they match said facts, they gain a similarity index of their own,
They are then compared with a book of law to check for the legality of the given argument, where an NLP checks for similar meanings 
 between the argument and the laws cited, and it too gets a similarity index as well as an impact index based on the fine/sentence the law it states has
 From my point of view the impact an argument has on the case can mostly only be realised by something of this sort, all of it is also multiplied with the number of arguments in total.

In [7]:
Argument_1_similarity=0.8  #The arguments similarity to the facts
Similarity_Citation=0.6
Punishment_For_Citation= 34  #There is a different scale required for this depending on the severity of the punishment with 100 being the death penalty or a very huge fine per say depending on case to case
Argument_1_impact= Argument_1_similarity*Similarity_Citation*Punishment_For_Citation
print("Impact of Argument 1 =",Argument_1_impact)
Disadvantage=Disadvantage-Argument_1_impact
print("Disadvantage before Defendant argument = ",Disadvantage)

Impact of Argument 1 = 16.32
Disadvantage before Defendant argument =  43.68


The defendants arguments are going to try to mostly just contradict the Plantiff's arguments so they are compared to the facts and then
They are compared to the defendant's arguments using the NLP, if there is a contradiction, the Argument gets its own values of similarity towards facts
picks up the similarity for citation from the previous section and the punishment for citation too and the impact is added to disadvantage.

In [8]:
Argument_contradiction_Similarity=0.7 #The argument's similarity to the facts
Argument_Contradiction_impact =Similarity_Citation*Punishment_For_Citation*Argument_contradiction_Similarity
Disadvantage=Disadvantage+Argument_Contradiction_impact
print("Impact of Argument 1 Contradiction =",Argument_Contradiction_impact)
print("Disadvantage after Defendant argument = ",Disadvantage)

Impact of Argument 1 Contradiction = 14.279999999999998
Disadvantage after Defendant argument =  57.959999999999994


Considering a case of law is mostly just a competition on who can use facts better 
even if the arguments are all contradicted , there is a chance still of the Defendant losing if the facts aren't correct.

All of these cases are going with the assumption that both the sides are given impactful and not bluffed or bloated arguments
As well as the case not being very moviesque in nature like we have seen some rare cases tend to be.

if the Plantiff ends with the disadvantage above 0, the defendant presumably wins the case.